# Homomorphic Encryption using Duet: Data Owner

### Setup

In [1]:
import syft as sy
import tenseal as ts
import pytest


### Start Duet Instance

In [2]:
duet = sy.launch_duet(loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...

♫♫♫ > CONNECTED!



### TenSeal Context

In [3]:
ts.Context()

ValueError: Invalid scheme type, use either SCHEME_TYPE.BFV or SCHEME_TYPE.CKKS

In [4]:
context = ts.Context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.global_scale = 2**40
context

### Encrypt the Data

In [27]:
v1 = [0, 1, 2, 3, 4]
v2 = [4, 3, 2, 1, 0]

enc_v1 = ts.ckks_vector(context, v1)
enc_v2 = ts.ckks_vector(context, v2)
(enc_v1, enc_v2)

(<tenseal.tensors.ckksvector.CKKSVector at 0x7f0d0ae028e0>,
 <tenseal.tensors.ckksvector.CKKSVector at 0x7f0d0ae02d00>)

In [28]:
result_add_local = enc_v1 + enc_v2
result_add_local.decrypt()

[3.999999999975049,
 4.000000000705923,
 3.9999999980663503,
 3.9999999993600746,
 3.999999999000849]

### Make Context and Encrypted Vectors Referenceable over Duet

In [29]:
# tag them so our partner can easily reference it
ctx_ptr = context.send(duet, pointable=True, tags=["context"])
enc_v1_ptr = enc_v1.send(duet, pointable=True, tags=["enc_v1"])
enc_v2_ptr = enc_v2.send(duet, pointable=True, tags=["enc_v2"])

In [30]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 03e4301ae9064a4c9c5b17bfc3a998fa>,[context],,<class 'tenseal.enc_context.Context'>
1,<UID: fe61d500b24148f9abc8e8a4fbdc915b>,[enc_v1],,<class 'tenseal.tensors.ckksvector.CKKSVector'>
2,<UID: cc12824db81c499b93b203575125a12d>,[enc_v2],,<class 'tenseal.tensors.ckksvector.CKKSVector'>


###  check requests

In [31]:
duet.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,[context],I would like to get the context,<UID: 478d76e6566d448a827fbd02429accba>,<UID: 03e4301ae9064a4c9c5b17bfc3a998fa>,<class 'tenseal.enc_context.Context'>
1,[enc_v1],I would like to get first vector,<UID: ca29f7890b3b4139843f174dab66bf07>,<UID: fe61d500b24148f9abc8e8a4fbdc915b>,<class 'tenseal.tensors.ckksvector.CKKSVector'>
2,[enc_v2],I would like to get second vector,<UID: d4928b79ef3d470d86aabd49d1f37446>,<UID: cc12824db81c499b93b203575125a12d>,<class 'tenseal.tensors.ckksvector.CKKSVector'>


### Approve the requests

In [32]:
duet.requests[0].accept()
duet.requests[0].accept()
duet.requests[0].accept()

In [33]:
duet.requests.pandas

""


### Get the computation results from store and decrypt them locally

In [36]:
# Validate the encrypted add
result_add = duet.store["result_add"].get(delete_obj=False)
result_add.link_context(context)

result_add

In [37]:
decrypted_result = result_add.decrypt()
regular_add = [v1 + v2 for v1, v2 in zip(v1, v2)]
assert pytest.approx(decrypted_result, abs=10**-3) == regular_add

decrypted_result = [round(r, 9) for r in decrypted_result]
print(f"Regular addition: {regular_add}\n"
      f"Decrypted addition: {decrypted_result}  ")

Regular addition: [4, 4, 4, 4, 4]
Decrypted addition: [4.0, 4.000000001, 3.999999998, 3.999999999, 3.999999999]  


In [39]:
# Validate the encrypted - plain add
result_iadd = duet.store["result_iadd"].get(delete_obj=False)
result_iadd.link_context(context)

decrypted_result = result_iadd.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v1 + v2 for v1, v2 in zip(v1, [10, 10, 10, 10, 10])]

decrypted_result

[10.000000000211422,
 10.99999999971608,
 11.999999998863512,
 13.000000000434593,
 13.999999998977376]

In [19]:
# Validate the encrypted subtraction

result_sub = duet.store["result_sub"].get(delete_obj=False)
result_sub.link_context(context)

decrypted_result = result_sub.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v1 - v2 for v1, v2 in zip(v1, v2)]

decrypted_result

[-3.9999999988133346,
 -2.0000000008387535,
 -4.566482747492273e-10,
 1.9999999997726194,
 3.999999999601287]

In [40]:
# Validate the encrypted multiplication

result_mul = duet.store["result_mul"].get(delete_obj=False)
result_mul.link_context(context)

decrypted_result = result_mul.decrypt()
regular_mult = [v1 * v2 for v1, v2 in zip(v1, v2)]
assert pytest.approx(decrypted_result, abs=10**-3) == regular_mult

decrypted_result = [round(r, 9) for r in decrypted_result]

print(f"Regular multiplication: {regular_mult}\n"
      f"Decrypted multiplication: {decrypted_result}")

Regular multiplication: [0, 3, 4, 3, 0]
Decrypted multiplication: [0.0, 3.000000402, 4.000000533, 3.000000399, -0.0]


In [21]:
# Validate the encrypted power

result_pow = duet.store["result_pow"].get(delete_obj=False)
result_pow.link_context(context)

decrypted_result = result_pow.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v ** 3 for v in v1]

decrypted_result

[2.6648017126262857e-10,
 1.000000801047932,
 8.000006433330313,
 27.00002172229583,
 64.00005148961425]

In [22]:
# Validate the encrypted negation

result_neg = duet.store["result_neg"].get(delete_obj=False)
result_neg.link_context(context)

decrypted_result = result_neg.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [-v for v in v1]

decrypted_result

[-8.498008963186976e-10,
 -0.9999999992972732,
 -1.999999999847142,
 -2.999999999827181,
 -3.999999999872858]

In [23]:
# Validate the encrypted polynomial evaluation for 1 + X^2 + X^3

result_poly = duet.store["result_poly"].get(delete_obj=False)
result_poly.link_context(context)

decrypted_result = result_poly.decrypt()
regular_poly = [1 + v**2 + v**3 for v in v1]
assert pytest.approx(decrypted_result, abs=10**-3) == regular_poly

decrypted_result = [round(r, 9) for r in decrypted_result]

print(f"Regular polynom: {regular_poly}\n"
      f"Decrypted polynom: {decrypted_result}")

Regular polynom: [1, 3, 13, 37, 81]
Decrypted polynom: [1.0, 3.000000933, 13.000006968, 37.000022929, 81.000053635]
